# 準備

## artのインストール

In [1]:
!pip3 install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 34.7 MB/s 
     |████████████████████████████████| 26.3 MB 92 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


## ライブラリのインポート

In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, Lambda
from keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()
from keras import backend as K

# ART
import art
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

## CIFAR10のロード・前処理

In [3]:
# CIFAR10のロード。
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# CIFAR10のラベル。
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

170508288/170498071 [==============================] - 11s 0us/step


In [4]:
# 正規化。
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# ラベルをOne-hot-vector化。
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Classifier(k-WTAなし)の作成



## モデル定義

In [5]:
# モデルの定義。
inputs = Input(shape=(32, 32, 3))
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
y = Dense(10, activation='softmax')(x)

model = Model(inputs, y)

# モデルのコンパイル。
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584

## 学習の実行

In [6]:
# 学習の実行。
model.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - ETA: 0s - loss: 2.2615 - accuracy: 0.1289

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


50000/50000 [==============================] - 72s 1ms/sample - loss: 2.2615 - accuracy: 0.1289 - val_loss: 2.1184 - val_accuracy: 0.1708
Epoch 2/30
50000/50000 [==============================] - 33s 669us/sample - loss: 1.9049 - accuracy: 0.2366 - val_loss: 1.7674 - val_accuracy: 0.3191
Epoch 3/30
50000/50000 [==============================] - 34s 672us/sample - loss: 1.5577 - accuracy: 0.4067 - val_loss: 1.4248 - val_accuracy: 0.4674
Epoch 4/30
50000/50000 [==============================] - 33s 669us/sample - loss: 1.2942 - accuracy: 0.5223 - val_loss: 1.2124 - val_accuracy: 0.5643
Epoch 5/30
50000/50000 [==============================] - 33s 665us/sample - loss: 1.1188 - accuracy: 0.5935 - val_loss: 1.1378 - val_accuracy: 0.5896
Epoch 6/30
50000/50000 [==============================] - 33s 655us/sample - loss: 0.9704 - accuracy: 0.6510 - val_loss: 0.9570 - val_accuracy: 0.6605
Epoch 7/30
50000/50000 [==============================] - 33s 660us/sample - loss: 0.8745 - accuracy: 0.690

## モデルの精度評価

In [7]:
model.evaluate(X_test, y_test)

[0.7760056887865067, 0.8071]

# Classifier(k-WTAあり)の作成

## k-WTAの作成

In [8]:
class KWTA(tf.keras.layers.Layer):
  def __init__(self, k, **kwargs) :
    super().__init__(**kwargs)
    self.k = k

  def call(self, x):
    topk = tf.math.top_k(x[0], k=self.k)   # 上位k個の値を抽出
    topk_min = K.min(topk.values)         # topkの最小値を抽出
    comp = tf.dtypes.cast(x >= topk_min, tf.float32)   # topkの最小値以上の値の部分を1, より小さい値の部分を0にしたTensorを生成
    return tf.math.multiply(x,comp)        # x * compより、topkの最小値以上の値のみそのまま、他は0になるようにする

## モデル定義

In [9]:
# モデルの定義。
inputs = Input(shape=(32, 32, 3))
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = GlobalAveragePooling2D()(x)

x = Dense(1024)(x)
x = KWTA(512)(x)
x = Dropout(0.25)(x)
x = Dense(1024)(x)
x = KWTA(512)(x)
x = Dropout(0.25)(x)
x = Dense(1024)(x)
x = KWTA(512)(x)
x = Dropout(0.25)(x)
y = Dense(10, activation='softmax')(x)

model_kwta = Model(inputs, y)

# モデルのコンパイル。
model_kwta.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_kwta.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 128)       1475

## 学習の実行

In [10]:
# 学習の実行。
model_kwta.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - ETA: 0s - loss: 2.1231 - accuracy: 0.1730

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


50000/50000 [==============================] - 34s 675us/sample - loss: 2.1231 - accuracy: 0.1730 - val_loss: 2.0295 - val_accuracy: 0.1896
Epoch 2/30
50000/50000 [==============================] - 33s 661us/sample - loss: 1.7949 - accuracy: 0.2780 - val_loss: 1.6298 - val_accuracy: 0.3726
Epoch 3/30
50000/50000 [==============================] - 33s 666us/sample - loss: 1.4510 - accuracy: 0.4415 - val_loss: 1.4035 - val_accuracy: 0.4756
Epoch 4/30
50000/50000 [==============================] - 33s 662us/sample - loss: 1.2203 - accuracy: 0.5501 - val_loss: 1.2976 - val_accuracy: 0.5211
Epoch 5/30
50000/50000 [==============================] - 33s 668us/sample - loss: 1.0618 - accuracy: 0.6119 - val_loss: 1.3140 - val_accuracy: 0.5151
Epoch 6/30
50000/50000 [==============================] - 33s 667us/sample - loss: 0.9372 - accuracy: 0.6628 - val_loss: 1.0317 - val_accuracy: 0.6314
Epoch 7/30
50000/50000 [==============================] - 34s 671us/sample - loss: 0.8111 - accuracy: 0.7

## モデルの精度評価

In [11]:
model_kwta.evaluate(X_test, y_test)

[0.7803472219467164, 0.8026]

# 敵対的攻撃

## 非標的型攻撃としてのFGSMによる摂動を作成
以下、この摂動を上乗せした画像のことを「敵対的画像」と呼ぶ。

In [12]:
# 入力データの特徴量の最小値・最大値を指定。
# 特徴量は0.0～1.0の範囲に収まるように正規化しているため、最小値は0.0、最大値は1.0とする。
min_pixel_value = 0.0
max_pixel_value = 1.0

### k-WTAなし

In [13]:
# モデルをART Keras Classifierでラップ。
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
# FGSMインスタンスの作成。
attack = FastGradientMethod(estimator=classifier, eps=0.10, targeted=False)
# 敵対的サンプルの生成（ベース画像はテストデータとする）。
X_adv = attack.generate(x=X_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


### k-WTAあり

In [14]:
classifier_kwta = KerasClassifier(model=model_kwta, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
attack_kwta = FastGradientMethod(estimator=classifier_kwta, eps=0.10, targeted=False)
X_adv_kwta = attack_kwta.generate(X_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


## 敵対的画像に対するモデルの精度評価

### k-WTAなし

In [15]:
model.evaluate(X_adv, y_test)

[7.944170335388184, 0.1025]

### k-WTAあり

In [16]:
model_kwta.evaluate(X_adv_kwta, y_test)

[8.029034226989745, 0.1237]